### Alphabet Soup Charity Optimization Model 3

In [20]:
%pip install keras-tuner --upgrade
# Import our dependencies
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn import __SKLEARN_SETUP__
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.utils import np_utils
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
csv_path = 'Resources/charity_data.csv'


#  Import and read the charity_data.csv.
application_df = pd.read_csv(csv_path)
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [21]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'], axis=1)
application_df.head()


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [22]:
# Determine the number of unique values in each column.
application_df.nunique()


APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [23]:
# Look at APPLICATION_TYPE value counts for binning
app_counts = application_df['APPLICATION_TYPE'].value_counts()
app_counts


T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [24]:
# Choose a cutoff value and create a list of application types to be replaced
application_types_to_replace = list(app_counts[app_counts < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(
        app, "Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()


T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [25]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df['CLASSIFICATION'].value_counts()
class_counts


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [26]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_counts_gt1 = class_counts.loc[class_counts > 1]
class_counts_gt1.head()


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [27]:
# Choose a cutoff value and create a list of classifications to be replaced
classifications_to_replace = list(class_counts[class_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(
        cls, "Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()


C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [28]:
# Convert categorical data to numeric with `pd.get_dummies`
application_numeric = pd.get_dummies(application_df)


In [29]:
# Split our preprocessed data into our features and target arrays
X = application_numeric.drop(['IS_SUCCESSFUL'], axis=1)
y = application_numeric['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=58)


In [30]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


## Compile, Train and Evaluate the Model

In [31]:
# Create a method that creats a new Sequential model with hyperparameter options

def create_model(hp):
    nn_model3 = tf.keras.models.Sequential()

    # Allow kerastuner to devide which activation function to use in hidden layers
    activation = hp.Choice('activation', ['sigmoid', 'tanh'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model3.add(tf.keras.layers.Dense(units=hp.Int('first_units', 
        min_value=1,
        max_value=5,
        step=5), activation=activation, input_dim=43))
    
    # Allow kerastuner to decide number of hidden layers and neurons to add to hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model3.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=5,
            step=5),
            activation=activation))
        
    nn_model3.add(tf.keras.layers.Dense(units=1, activation='relu'))

    # Compile the model
    nn_model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return nn_model3

# Create kerastuner
import keras_tuner as kt

tuner = kt.Hyperband(create_model,
                     objective='val_accuracy',
                     max_epochs=5,
                     hyperband_iterations=2)


# Search for the best hyperparameters
tuner.search(X_train_scaled, y_train, epochs=5, validation_data=(X_test_scaled, y_test))

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=3)[0]

# Build the model with the best hyperparameters
model_3 = create_model(best_hps)

# Train the model
fit_model = model_3.fit(X_train_scaled, y_train, epochs=5)


INFO:tensorflow:Reloading Tuner from .\untitled_project\tuner0.json
INFO:tensorflow:Oracle triggered exit
Epoch 1/5
804/804 [==============================] - 30s 9ms/step - loss: 0.9112 - accuracy: 0.6766
Epoch 2/5
804/804 [==============================] - 5s 7ms/step - loss: 0.5725 - accuracy: 0.7222
Epoch 3/5
804/804 [==============================] - 5s 6ms/step - loss: 0.5787 - accuracy: 0.7138
Epoch 4/5
804/804 [==============================] - 5s 7ms/step - loss: 0.5756 - accuracy: 0.7115
Epoch 5/5
804/804 [==============================] - 6s 8ms/step - loss: 0.5630 - accuracy: 0.7274


In [13]:
# Import the kerastuner library
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model,
    objective='val_accuracy',
    max_epochs=5,
    hyperband_iterations=2)



INFO:tensorflow:Reloading Tuner from .\untitled_project\tuner0.json


In [32]:
# Run the kerastuner search for the best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled, y_test))


Search: Running Trial #147

Value             |Best Value So Far |Hyperparameter
sigmoid           |tanh              |activation
1                 |26                |first_units
1                 |5                 |num_layers
16                |6                 |units_0
16                |21                |units_1
11                |6                 |units_2
26                |16                |units_3
1                 |26                |units_4
4                 |10                |tuner/epochs
0                 |4                 |tuner/initial_epoch
1                 |2                 |tuner/bracket
0                 |2                 |tuner/round

Epoch 1/4
800/804 [============================>.] - ETA: 0s - loss: 0.6667 - accuracy: 0.6138

In [17]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values


{'activation': 'tanh',
 'first_units': 26,
 'num_layers': 5,
 'units_0': 6,
 'units_1': 21,
 'units_2': 6,
 'units_3': 16,
 'units_4': 26,
 'tuner/epochs': 10,
 'tuner/initial_epoch': 4,
 'tuner/bracket': 2,
 'tuner/round': 2,
 'tuner/trial_id': '0098'}

In [19]:
# Evaluate the best model against full test data
best_model = tuner.get_best_models(3)
for model in best_model:
    model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f'Loss: {model_loss}, Accuracy: {model_accuracy}')

AttributeError: 'list' object has no attribute 'evaluate'

In [ ]:
# Export our model to HDF5 file
# Define the filename
filename = 'H5_Files/AlphabetSoupCharity_Optimization3.h5'


# Save the model to a HDF5 file
best_model.save(filename)
